In [3]:
import os
import re
import pandas as pd

pasta = "C:/Users/fcsantos/OneDrive - BERNHOEFT CONTADORES ASSOCIADOS LTDA/_Inteligência_Empresarial/Atividades_Felipe/Evertrack/dados_brutos"
pasta_saida = "C:/Users/fcsantos/OneDrive - BERNHOEFT CONTADORES ASSOCIADOS LTDA/_Inteligência_Empresarial/Atividades_Felipe/Evertrack/dados_brutos_concatenados"

# Encontre todos os arquivos .csv na pasta
arquivos = [f for f in os.listdir(pasta) if f.endswith('.csv')]

lista_de_dataframes = []
data_inicial = None
data_final = None

total_linhas_originais = 0
linhas_com_valores_em_branco = 0

# Leia cada arquivo e adicione-o à lista de dataframes
for arquivo in arquivos:
    df = pd.read_csv(os.path.join(pasta, arquivo), encoding='ISO-8859-1')
    
    # Contabilize o total de linhas originais
    total_linhas_originais += len(df)
    
    # Contabilize as linhas com valores em branco
    linhas_com_valores_em_branco += df.isnull().sum().sum()
    
    lista_de_dataframes.append(df)

    # Encontre as datas no nome do arquivo
    datas = re.findall(r'\d{4}_\d{2}_\d{2}', arquivo)
    if datas:
        # Converta para o formato aaaa-mm-dd
        datas = [data.replace('_', '-') for data in datas]

        # Atualize a data inicial e final conforme necessário
        if not data_inicial or datas[0] < data_inicial:
            data_inicial = datas[0]
        if not data_final or datas[-1] > data_final:
            data_final = datas[-1]

# Concatene todos os dataframes em um único dataframe
df_final = pd.concat(lista_de_dataframes)

# Verifique se alguma linha foi perdida
total_linhas_final = len(df_final)
if total_linhas_originais == total_linhas_final:
    print("Nenhuma linha foi perdida na concatenação.")
else:
    print(f"{total_linhas_originais - total_linhas_final} linhas foram perdidas na concatenação.")

# Verifique se existem linhas com valores em branco
if linhas_com_valores_em_branco > 0:
    print(f"Há {linhas_com_valores_em_branco} células com valores em branco nos dados originais.")

# Crie o nome do arquivo de saída
nome_saida = f"dados_brutos_concatenados_{data_inicial}_a_{data_final}.csv"
caminho_saida = os.path.join(pasta_saida, nome_saida)

# Salve o dataframe final
df_final.to_csv(caminho_saida, index=False, encoding='ISO-8859-1')


Nenhuma linha foi perdida na concatenação.


In [10]:
import pandas as pd
import datetime

# Carregar o arquivo de dados
arquivo_dados = "C:\\Users\\fcsantos\\OneDrive - BERNHOEFT CONTADORES ASSOCIADOS LTDA\\_Inteligência_Empresarial\\Atividades_Felipe\\Evertrack\\dados_brutos_concatenados\\dados_brutos_concatenados_2023-04-18_a_2023-05-09.csv"
df = pd.read_csv(arquivo_dados, encoding='ISO-8859-1', sep=';')

# Remover espaços extras no nome das colunas
df.columns = df.columns.str.strip()

# Corrigir o nome das colunas 'Duração' e 'Aplicação'
df.columns = df.columns.str.replace('DuraÃ§Ã£o', 'Duração')
df.columns = df.columns.str.replace('AplicaÃ§Ã£o', 'Aplicação')
df.columns = df.columns.str.replace('ï»¿Time', 'Time')

# Converter a coluna 'Duração' em timedelta
df['Duração'] = pd.to_timedelta(df['Duração'])

# Filtrar as atividades com valor "Neutra" na coluna "Tipo da categoria"
df_neutras = df[df['Tipo da categoria'] == 'Neutra']

# Agrupar por 'Time' e 'Aplicação', somando a 'Duração'
df_neutras = df_neutras.groupby(['Time', 'Aplicação'])['Duração'].sum().reset_index()

# Ordenar por 'Duração' em ordem decrescente
df_neutras = df_neutras.sort_values(by='Duração', ascending=False)

# Salvar a análise em um arquivo Excel
data_atual = datetime.datetime.now().strftime("%Y-%m-%d")
arquivo_saida = f"C:\\Users\\fcsantos\\OneDrive - BERNHOEFT CONTADORES ASSOCIADOS LTDA\\_Inteligência_Empresarial\\Atividades_Felipe\\Evertrack\\dados_brutos_concatenados\\analise_atividades_neutras_{data_atual}.xlsx"
df_neutras.to_excel(arquivo_saida, index=False)

print("Análise salva com sucesso.")


Análise salva com sucesso.
